### Parámetros comunes

In [1]:
datasets = {"restaurants":["gijon", "barcelona", "madrid", "paris", "newyorkcity"],
            "pois":["barcelona", "madrid", "paris", "newyorkcity", "london"],
            "amazon":["fashion", "digital_music"]}

models = ["ATT2ITM", "BOW2ITM", "USEM2ITM", "BERT2ITM"]
models = ["MOSTPOP2ITM"]

### Evaluar nuestros modelos solo en los casos de cold-start

Esto es una solución "temporal" para poder obtener los resultados del baseline "MostPopular" para el cold-start.
El problema es que no hay que entrenarlos y no hay un "best_model", por tanto el resto del flujo no funciona.

In [ ]:
from src.experiments.Common import load_best_model
import pandas as pd

cold_start_res = []

for dataset, subsets in datasets.items():
    for subset in subsets:
        for model in models:
            # Cargar configuración mejor modelo
            model_class = load_best_model(model=model, dataset=dataset, subset=subset)
            # Cargar el modelo entrenado
            model_class.train(dev=False, save_model=True)
            # Eliminar usuarios que aparecen en TRAIN
            train_dev_users = model_class.DATASET.DATA["TRAIN_DEV"].userId.unique()
            # original_test_len = len(model_class.DATASET.DATA["TEST"])
            model_class.DATASET.DATA["TEST"] = model_class.DATASET.DATA["TEST"][~model_class.DATASET.DATA["TEST"]["userId"].isin(train_dev_users)]
            model_class.DATASET.DATA["TEST"] = model_class.DATASET.DATA["TEST"].drop_duplicates(subset=["userId", "id_item"], keep='last', inplace=False)
            # Evaluar en el conjunto de test
            test_ret = model_class.evaluate(test=True)
            cold_start_res.append([dataset, subset, model]+test_ret.values.tolist()[0][1:]+[len(model_class.DATASET.DATA["TEST"])])

results = pd.DataFrame(cold_start_res, columns=["Set", "Subset", "Model"] + test_ret.columns.tolist()[1:]+["cold_items"])
# cold_start_res.to_excel("cold_start_evaluation.xlsx")

### Generar tabla Latex

In [3]:
# results = pd.read_excel("/media/nas/pperez/code/TAVtext/cold_start_evaluation.xlsx")
# Poner nombres decentes para el artículo
results["Set"].replace({"restaurants": "TAV-RSTS", "pois":"TAV-POIS", "amazon":"AM"}, inplace=True)
results["Subset"].replace({"digital_music": "Music", "fashion":"Fashion", "gijon": "Gijón", "barcelona":"Barcelona", "madrid": "Madrid", "paris": "Paris", "newyorkcity": "New York", "london": "London"}, inplace=True)
results["Model"].replace({"USEM2ITM":"USEM", "BERT2ITM":"BERT", "BOW2ITM":"TRecX", "ATT2ITM": "AITRecX"}, inplace=True)

In [4]:
metrics = ["Recall@5", "Recall@10", "NDCG@10"]
models = ['TRecX', 'AITRecX', 'USEM', 'BERT']

result_df = []

for metric in metrics:
    mres = results.pivot_table(index=[ "Set", "Subset"], columns=["Model"])[metric].reset_index()
    mres["Metric"] = metric
    result_df.extend(mres.values)

result_df = pd.DataFrame(result_df, columns=mres.columns)

# Especificar el orden de los datos
result_df['Set'] = pd.Categorical(result_df['Set'], ["AM", "TAV-POIS", "TAV-RSTS"])
result_df['Subset'] = pd.Categorical(result_df['Subset'], ["Music", "Fashion", "Gijón", "Barcelona", "Madrid", "New York", "Paris", "London"])
result_df['Metric'] = pd.Categorical(result_df['Metric'], metrics)

result_df = result_df.pivot_table(index=["Set", "Subset", "Metric"])
print(result_df[models].to_latex())
result_df.to_excel("cold_start_results.xlsx")

\begin{tabular}{lllrrrr}
\toprule
         &       & Model &     TRecX &   AITRecX &      USEM &      BERT \\
Set & Subset & Metric &           &           &           &           \\
\midrule
AM & Music & Recall@5 &  0.419999 &  0.508491 &  0.065422 &  0.065625 \\
         &       & Recall@10 &  0.483053 &  0.572762 &  0.095934 &  0.095866 \\
         &       & NDCG@10 &  0.373396 &  0.452882 &  0.052101 &  0.051918 \\
         & Fashion & Recall@5 &  0.470546 &  0.553146 &  0.058260 &  0.056901 \\
         &       & Recall@10 &  0.558066 &  0.641442 &  0.078651 &  0.084995 \\
         &       & NDCG@10 &  0.401084 &  0.477773 &  0.034589 &  0.036435 \\
TAV-POIS & Barcelona & Recall@5 &  0.919263 &  0.935552 &  0.922805 &  0.921034 \\
         &       & Recall@10 &  0.955737 &  0.966006 &  0.950779 &  0.953612 \\
         &       & NDCG@10 &  0.847057 &  0.865142 &  0.861960 &  0.858370 \\
         & Madrid & Recall@5 &  0.912214 &  0.923664 &  0.914838 &  0.923664 \\
         &       

/tmp/ipykernel_3887499/3593426627.py:19: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(result_df[models].to_latex())


### Gráficas de evolución de nuestros modelos y baselines en función del usuarios en Train
No parece verse mejora clara en los baselines, por tanto no se mostrarán gráficas.

In [ ]:
# Importar pandas
import matplotlib.pyplot as plt
import pandas as pd
import os

top = 5
cumulate = True

datasets = {"restaurants":["gijon", "barcelona", "madrid", "paris", "newyorkcity"],
            "pois":["barcelona", "madrid", "paris", "newyorkcity", "london"],
            "amazon":["fashion", "digital_music"]}

datasets = {"restaurants":["barcelona"]}

models = ["ATT2ITM", "BOW2ITM", "USEM2ITM"]

best_model = pd.read_csv("models/best_models.csv")

for dataset, subsets in datasets.items():
    for subset in subsets:
        fig, ax = plt.subplots(figsize=(10, 5))
        ax2 = ax.twinx()
        subset_results = pd.DataFrame(range(top+1), columns=["train_dev_items"])
        
        baseline_path = f"models/Baselines/{dataset}/{subset}/user_results.csv"
        if os.path.exists(baseline_path): 
            baselines = pd.read_csv(baseline_path).rename(columns={"cold":"train_dev_items"})
            baselines_names = baselines.columns.to_list(); baselines_names.remove("userId"); baselines_names.remove("train_dev_items")
            baselines = baselines.groupby("train_dev_items")[baselines_names].mean().reset_index()
            subset_results = subset_results.merge(baselines, how="left")

            for bl in baselines_names: # ["GridSearch_BPR", "GridSearch_EASEᴿ", "MostPop"]: 
                if cumulate: subset_results[bl] = subset_results[bl].expanding().mean()
                ax.plot(subset_results["train_dev_items"], subset_results[bl], "--", label=bl)

        else: raise ValueError

        for model in models:
            md5 = best_model.loc[(best_model.dataset == dataset) & (best_model.subset == subset) & (best_model.model == model)]["model_md5"].values[0]
            results_path = f"models/{model}/{dataset}/{subset}/{md5}/final_user_eval.csv"
            if os.path.exists(results_path):
                results = pd.read_csv(results_path)
                results = results.groupby("cold").agg(mn=("NDCG", "mean"), n_samples=("NDCG", "count")).reset_index().rename(columns={"cold":"train_dev_items", "mn":model})
                if cumulate: results[model] = results[model].expanding().mean()
                subset_results = subset_results.merge(results, how="left")
                ax.plot(subset_results["train_dev_items"], subset_results[model], "-", label=model)
            else:
                raise ValueError
        
        subset_results["n_samples"] = subset_results["n_samples"].fillna(0)

        ax2.plot(subset_results["train_dev_items"], subset_results["n_samples"], ":",label="")

        ax.set_ylim([0, 1])
        ax2.set_ylim([0, subset_results["n_samples"].max()])
        ax.set_ylabel('AVG NDCG')
        plt.title(f"{dataset.title()} - {subset.title()}")
        ax.set_xlabel("Number of reviews in Train set")
        # ax.set_xticks(range(top+1))
        # ax.set_xlim([0, top])
        ax2.set_ylabel('Number of Test cases')

        ax.legend()
        plt.show()